In [23]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
from cellpose.io import imread
from pathlib import Path
import os
import Modules
# List of Files
basedir = os.path.join(Path.cwd(), "Images") # c:\Users\birat\Documents\Internship_Code\Images
files = io.get_image_files(basedir, "")
files = files[3::]


In [24]:
from PIL import Image
# Makes the directy to point to the seg files
Segdir = []
for f in files: 
    Segdir.append((os.path.join(basedir, f) + "_seg.npy").replace(".png", ""))

In [25]:
def getROI(Segdir, all = False): # Requires directory to the segmentation.npy files 
    from statistics import mode
    max = []
    for d in Segdir: 
        dat = np.load(d, allow_pickle=True).item()
        max.append(np.amax(dat['masks']))
    if all == False: 
        return mode(max)
    elif all == True: 
        return max

In [26]:
def evalOne(filename, flowe = 0.4, mm): 
    img = imread(filename)
    modeldir = os.path.join(Path.cwd(),"Images","models","Old_Model") 
    model = models.CellposeModel(gpu = True, pretrained_model = modeldir)
    masks, flows, styles = model.eval(img, diameter=None, channels=[0,0], flow_threshold=flowe)
    # save results so you can load in gui
    io.masks_flows_to_seg(img, mm, flows, styles, filename, [0,0])

def LowerFlow(Segdir, imgdir): # Takes in the directory of the thingy 
    count = 0  
    roiList = getROI(Segdir, all = True)
    roi = getROI(Segdir)
    for ROI in roiList:
        print("ROI: ", ROI) 
        if ROI != roi: # If not the mode ROI, 
            flowe = 0.4
            badROI = ROI 
            count = 0 
            while badROI != roi or count == 4:
                if ROI < roi:
                    flowe = flowe + 0.05
                    print("Increased Flow: ", flowe)
                elif ROI > roi: 
                    flowe = flowe - 0.05
                    print("Decreased Flow: ", flowe)

                count += 1
                print(count)
                evalOne(imgdir[count], flowe ) # Evaluate till it is 
                print("evaluated")
                
                newROIList = getROI(Segdir, True)
                badROI = newROIList[count]
                count +=1
    print("completed")

In [1]:
segFiles = Modules.getFileDir(seg=True)
imgFiles = Modules.getFileDir()
segFiles = segFiles[3::]
a = getROI(segFiles, all = True)

# segDir = segFiles[-3]
# imgDir = imgFiles[-3]
# LowerFlow(segFiles, imgFiles)



NameError: name 'Modules' is not defined

In [17]:
from cellpose import plot, utils

count = 0 
for d in dir: 
    dat = np.load(d, allow_pickle=True).item()
    outlines = utils.outlines_list(dat['masks'])
    fig = plt.figure()
    fig.set_size_inches((5, 5))
    for o in outlines:
        plt.plot(o[:,0], o[:,1], color='black')
    ax = plt.Axes(fig, [0, 0, 1, 1])

    plt.axis('off')
    plt.margins(x=0, y =0)
    savedir = ".\\Plots\\" + str(count)
    count += 1 
    plt.savefig(savedir)

    # Plots the figures in grayscale

TypeError: 'builtin_function_or_method' object is not iterable